In [1]:
import numpy as np

# Heston Model

In [13]:
def get_initial_asset_price():
    while True:
        S = input('Enter the initial asset price : ')
        try:
            initial_asset_price = int(S)
            return initial_asset_price
        except ValueError:
            print("Invalid number type. Please enter an integer number.")

def get_initial_volatility():
    while True:
        vol = input('Enter the Volatility (between 0 and 1) : ')
        try:
            volatility = float(vol)
            if 0 <= volatility <= 1:
                return volatility
            else:
                print("Invalid value. Please enter a number between 0 and 1.")
        except ValueError:
            print("Invalid number type. Please enter a decimal number between 0 and 1.")

def get_long_term_variance():
    while True:
        var_long = input('Enter the Long Term Variance (between 0 and 1) : ')
        try:
            var_long_term = float(var_long)
            if 0 <= var_long_term <= 1:
                return var_long_term
            else:
                print("Invalid value. Please enter a number between 0 and 1.")
        except ValueError:
            print("Invalid number type. Please enter a decimal number between 0 and 1.")
            
def get_rate_of_reversion():
    while True:
        rate = input('Enter the rate of reversion of variance (between -1 and 1): ')
        try:
            rate_of_reversion = float(rate)
            if -1 <= rate_of_reversion <= 1:
                return rate_of_reversion
            else:
                print("Invalid value. Please enter a number between -1 and 1.")
        except ValueError:
            print("Invalid number type. Please enter a decimal number between -1 and 1.")

def get_vol_of_vol():
    while True:
        v_o_v = input('Enter the Volatility of Volatility (between 0 and 1): ')
        try:
            vol_of_vol = float(v_o_v)
            if 0 <= vol_of_vol <= 1:
                return vol_of_vol
            else:
                print("Invalid value. Please enter a number between 0 and 1.")
        except ValueError:
            print("Invalid number type. Please enter a decimal number between 0 and 1.")

def get_correlation():
    while True:
        cor = input('Enter the correlation between the two processes (between -1 and 1): ')
        try:
            correlation = float(cor)
            if -1 <= correlation <= 1:
                return correlation
            else:
                print("Invalid value. Please enter a number between -1 and 1.")
        except ValueError:
            print("Invalid number type. Please enter a decimal number between -1 and 1.")

def get_rf():
    while True:
        rate = input('Enter the Risk-Free rate (between 0 and 1): ')
        try:
            rate = float(rate)
            if 0 <= rate <= 1:
                return rate
            else:
                print("Invalid value. Please enter a number between 0 and 1.")
        except ValueError:
            print("Invalid number type. Please enter a decimal number between 0 and 1.")

def get_maturity():
    while True:
        M = input('Enter the Maturity (in years): ')
        try:
            maturity = float(M)
            if maturity > 0:
                return maturity
            else:
                print("Invalid value. Please enter a positive number.")
        except ValueError:
            print("Invalid number type. Please enter a valid number.")

def get_time_step():
    while True:
        steps = input('Enter the number of time steps (between 0 and 1000): ')
        try:
            steps = int(steps)
            if 0 <= steps <= 1000:
                return steps
            else:
                print("Invalid value. Please enter a number between 0 and 1000.")
        except ValueError:
            print("Invalid number type. Please enter a integer number between 0 and 1000.")
    
def get_paths():
    while True:
        paths = input('Enter the number of paths (between 0 and 10000): ')
        try:
            paths = int(paths)
            if 0 <= paths <= 10000:
                return paths
            else:
                print("Invalid value. Please enter a number between 0 and 10000.")
        except ValueError:
            print("Invalid number type. Please enter a integer number between 0 and 10000.")

In [14]:
S0 = get_initial_asset_price()  
v0 = get_initial_volatility()      
theta = get_long_term_variance()   
kappa = get_rate_of_reversion()  
sigma_v = get_vol_of_vol()  
rho = get_correlation()     
r = get_rf()       
T = get_maturity()        
N = get_time_step()      
M = get_paths()     

In [15]:
print("Parameter Values:")
print("-----------------")
print(f"Initial Asset Price (S0):     {S0} $")
print(f"Initial Volatility (v0):      {v0*100} %")
print(f"Long-term Variance (theta):   {theta*100} %")
print(f"Rate of Reversion (kappa):    {kappa*100} %")
print(f"Volatility of Volatility:     {sigma_v*100} %")
print(f"Correlation (rho):            {rho}")
print(f"Risk-free Interest Rate (r):  {r*100} %")
print(f"Maturity (T):                 {T} years")
print(f"Number of Time Steps (N):     {N}")
print(f"Number of Paths (M):          {M}")

Parameter Values:
-----------------
Initial Asset Price (S0):     100
Initial Volatility (v0):      0.04
Long-term Variance (theta):   0.04
Rate of Reversion (kappa):    0.5
Volatility of Volatility:     0.3
Correlation (rho):            -0.7
Risk-free Interest Rate (r):  0.05
Maturity (T):                 1.0
Number of Time Steps (N):     1000
Number of Paths (M):          10000
